[![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb)

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running on AWS.


## Prerequisites

Install dependencies.

In [1]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pandas==2.0.3

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [ ]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
pc = Pinecone(api_key=api_key)

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [2]:
# Giving our index a name
index_name = "hello-pinecone"

In [3]:
pc.list_indexes().names()

[]

In [4]:
# Delete the index, if an index of the same name already exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

We define a `spec` object to define the cloud region we'd like to create an index within. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Creating a Pinecone Index.

In [5]:
import time

dimensions = 3
pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine"
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [6]:
index = pc.Index(index_name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [7]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [8]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{'upserted_count': 2}

In [9]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2}},
 'total_vector_count': 2}

In [11]:
index.query(
    vector=[2., 2., 2.],
    top_k=1,
    include_values=True
) # returns top_k matches

{'matches': [{'id': 'A', 'score': 1.0, 'values': [1.0, 1.0, 1.0]}],
 'namespace': ''}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [12]:
pc.delete_index(index_name)

---